In [1]:
import gym
import random
import csv
import time
from itertools import product
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy


# these are the hyperparameters we can edit
learning_rates = [0.1, 0.01, 0.001]
n_steps = [512]
batch_sizes = [128]
gammas = [0.9, 0.95, 0.99, 0.999]
gae_lambdas = [0.95, 0.90, 0.85]
clip_ranges = [0.1, 0.2, 0.3]
ent_coefs = [0.0, 0.1, 0.01]
epsilons = [1e-5, 1e-6]

#648 combinations x 45.5 seconds should be 29484

best_hyperparams = None
best_score = -float('inf')

with open('hyperparam_results_grid_search_bipedal_hardcore_100,000.csv', 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['learning_rate', 'n_steps', 'batch_size', 'gamma', 'gae_lambda', 'clip_range', 'ent_coef', 'epsilon', 'mean_reward', 'training_time', 'standard deviation'])

    for hyperparams in product(learning_rates, n_steps, batch_sizes, gammas, gae_lambdas, clip_ranges, ent_coefs, epsilons):
        learning_rate, n_steps, batch_size, gamma, gae_lambda, clip_range, ent_coef, epsilon = hyperparams

        from gym.envs.box2d import BipedalWalker

        class BipedalWalkerHardcore(BipedalWalker):
            hardcore = True

            def __init__(self, *args, **kwargs):
                super().__init__(*args, **kwargs)


        environment_name = 'BipedalWalkerHardcore-v3'
        gym.register(
            id=environment_name,
            entry_point='__main__:BipedalWalkerHardcore',
            max_episode_steps=1600,
            reward_threshold=300,
        )

        env = gym.make(environment_name)


        model = PPO('MlpPolicy', env,
                    learning_rate=learning_rate,
                    n_steps=n_steps,
                    batch_size=batch_size,
                    gamma=gamma,
                    gae_lambda=gae_lambda,
                    clip_range=clip_range,
                    ent_coef=ent_coef,
                    clip_range_vf=epsilon,
                    verbose=1)

        print("Training with hyperparameters:", hyperparams)

        start_time = time.time()
        model.learn(total_timesteps=100000) #100 thousand steps
        end_time = time.time()

        training_time = end_time - start_time

        mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
        print("Mean reward: {:.2f} +/- {:.2f}".format(mean_reward, std_reward))

        if mean_reward > best_score:
            best_hyperparams = hyperparams
            best_score = mean_reward
            print("New best hyperparameters found:", best_hyperparams)
            print("New best score found:", best_score)

        with open('hyperparam_results_grid_search_bipedal_hardcore_100,000.csv', 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow((learning_rate, n_steps, batch_size, gamma, gae_lambda, clip_range, ent_coef, epsilon, mean_reward,training_time, std_reward))

    print("Best hyperparameters:", best_hyperparams)
    print("Best score:", best_score)


/home/csacoa/anaconda3/envs/rl2/lib/python3.8/site-packages/gym/envs/registration.py:216: UserWarning: WARN: Overriding environment BipedalWalkerHardcore-v3
  logger.warn("Overriding environment {}".format(id))


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Training with hyperparameters: (0.1, 512, 128, 0.9, 0.95, 0.1, 0.0, 1e-05)
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 74.8     |
|    ep_rew_mean     | -112     |
| time/              |          |
|    fps             | 1524     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 512      |
---------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 74.8      |
|    ep_rew_mean          | -112      |
| time/                   |           |
|    fps                  | 1388      |
|    iterations           | 2         |
|    time_elapsed         | 0         |
|    total_timesteps      | 1024      |
| train/                  |           |
|    approx_kl            | 255.04086 |
|    clip_fraction        | 0.974     |
|    clip_r

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 125       |
|    ep_rew_mean          | -120      |
| time/                   |           |
|    fps                  | 1282      |
|    iterations           | 11        |
|    time_elapsed         | 4         |
|    total_timesteps      | 5632      |
| train/                  |           |
|    approx_kl            | 126.10528 |
|    clip_fraction        | 0.954     |
|    clip_range           | 0.1       |
|    clip_range_vf        | 1e-05     |
|    entropy_loss         | -11.3     |
|    explained_variance   | -8.26e-05 |
|    learning_rate        | 0.1       |
|    loss                 | 328       |
|    n_updates            | 100       |
|    policy_gradient_loss | 0.193     |
|    std                  | 11.6      |
|    value_loss           | 642       |
---------------------------------------
---------------------------------------
| rollout/                |           |


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 233       |
|    ep_rew_mean          | -125      |
| time/                   |           |
|    fps                  | 1254      |
|    iterations           | 20        |
|    time_elapsed         | 8         |
|    total_timesteps      | 10240     |
| train/                  |           |
|    approx_kl            | 7727.2114 |
|    clip_fraction        | 0.967     |
|    clip_range           | 0.1       |
|    clip_range_vf        | 1e-05     |
|    entropy_loss         | -7.24     |
|    explained_variance   | -2.38e-07 |
|    learning_rate        | 0.1       |
|    loss                 | 6.47      |
|    n_updates            | 190       |
|    policy_gradient_loss | 0.172     |
|    std                  | 1.71      |
|    value_loss           | 12.7      |
---------------------------------------
---------------------------------------
| rollout/                |           |


--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 315      |
|    ep_rew_mean          | -128     |
| time/                   |          |
|    fps                  | 1255     |
|    iterations           | 29       |
|    time_elapsed         | 11       |
|    total_timesteps      | 14848    |
| train/                  |          |
|    approx_kl            | 3541.012 |
|    clip_fraction        | 0.954    |
|    clip_range           | 0.1      |
|    clip_range_vf        | 1e-05    |
|    entropy_loss         | -6.49    |
|    explained_variance   | 0        |
|    learning_rate        | 0.1      |
|    loss                 | 281      |
|    n_updates            | 280      |
|    policy_gradient_loss | 0.145    |
|    std                  | 1.29     |
|    value_loss           | 560      |
--------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean         

...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 221         |
|    ep_rew_mean          | -124        |
| time/                   |             |
|    fps                  | 1164        |
|    iterations           | 191         |
|    time_elapsed         | 83          |
|    total_timesteps      | 97792       |
| train/                  |             |
|    approx_kl            | 0.008881306 |
|    clip_fraction        | 0.51        |
|    clip_range           | 0.1         |
|    clip_range_vf        | 1e-06       |
|    entropy_loss         | -19.4       |
|    explained_variance   | -1.19e-07   |
|    learning_rate        | 0.1         |
|    loss                 | 1.59e+03    |
|    n_updates            | 1900        |
|    policy_gradient_loss | 0.0184      |
|    std                  | 36.6        |
|    value_loss           | 3.21e+03    |
-----------------------------------------
----------------------------------

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 152      |
|    ep_rew_mean          | -118     |
| time/                   |          |
|    fps                  | 1108     |
|    iterations           | 5        |
|    time_elapsed         | 2        |
|    total_timesteps      | 2560     |
| train/                  |          |
|    approx_kl            | 9.955282 |
|    clip_fraction        | 0.952    |
|    clip_range           | 0.1      |
|    clip_range_vf        | 1e-05    |
|    entropy_loss         | -48.9    |
|    explained_variance   | -0.0562  |
|    learning_rate        | 0.1      |
|    loss                 | 54.7     |
|    n_updates            | 40       |
|    policy_gradient_loss | 0.116    |
|    std                  | 5.01e+05 |
|    value_loss           | 189      |
--------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 232         |
|    ep_rew_mean          | -120        |
| time/                   |             |
|    fps                  | 1078        |
|    iterations           | 14          |
|    time_elapsed         | 6           |
|    total_timesteps      | 7168        |
| train/                  |             |
|    approx_kl            | 0.013386563 |
|    clip_fraction        | 0.564       |
|    clip_range           | 0.1         |
|    clip_range_vf        | 1e-05       |
|    entropy_loss         | -119        |
|    explained_variance   | -8.11e-06   |
|    learning_rate        | 0.1         |
|    loss                 | -5.86       |
|    n_updates            | 130         |
|    policy_gradient_loss | 0.046       |
|    std                  | 3.56e+12    |
|    value_loss           | 12.1        |
-----------------------------------------
----------------------------------

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 200       |
|    ep_rew_mean          | -121      |
| time/                   |           |
|    fps                  | 1068      |
|    iterations           | 23        |
|    time_elapsed         | 11        |
|    total_timesteps      | 11776     |
| train/                  |           |
|    approx_kl            | 1.0782106 |
|    clip_fraction        | 0.785     |
|    clip_range           | 0.1       |
|    clip_range_vf        | 1e-05     |
|    entropy_loss         | -145      |
|    explained_variance   | -0.0147   |
|    learning_rate        | 0.1       |
|    loss                 | 280       |
|    n_updates            | 220       |
|    policy_gradient_loss | 0.0791    |
|    std                  | 6.73e+15  |
|    value_loss           | 585       |
---------------------------------------
-----------------------------------------
| rollout/                |           

ValueError: Expected parameter loc (Tensor of shape (128, 4)) of distribution Normal(loc: torch.Size([128, 4]), scale: torch.Size([128, 4])) to satisfy the constraint Real(), but found invalid values:
tensor([[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, 